### Quickstart

#### Install GraphRAG


In [1]:
! pip install graphrag

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Using cached matplotlib-3.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.5 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 5.7 MB/s eta 0:00:00
  Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached ji

#### Build the index

#### 构建索引


#### Now we need to set up a data project and some initial configuration. Let's get started. We use the default configuration mode, you can customize it as needed using a configuration file (which we recommend) or environment variables.
#### 现在我们需要设置一个数据项目和一些初始配置。让我们开始设置吧。我们使用默认配置模式，您可以根据需要使用配置文件（我们推荐）或环境变量进行自定义。

In [2]:
!rm -rf ./coplay_analysis_graph/output
!rm -rf ./coplay_analysis_graph/prompts
!rm -rf ./coplay_analysis_graph/settings.yaml

To initialize your workspace, we first run the `graphrag.index --init` command. Here a directory called .coplay_analysis_graph` is configured, so we can run the following command:
为了初始化你的工作区，我们首先运行 `graphrag.index --init` 命令。这里配置了一个名为 .coplay_analysis_graph` 的目录，因此我们可以运行以下命令：

In [3]:

import argparse
import os
from pathlib import Path
 


root_dir =  Path().cwd() /"coplay_analysis_graph" 

!mkdir $root_dir
!python -m graphrag.index --init --root "$root_dir"

Initializing project at 
/media/gpt4-pdf-chatbot-langchain/glm-cookbook/demo/graphrag/coplay_analysis_gra
ph
⠋ GraphRAG Indexer 

### Set workspace variables
First, let's make sure the required environment variables are set. For more information about these environment variables and the available environment variables, see the variables documentation.

Here you need to set the platform's api_key, which can be obtained from `https://open.bigmodel.cn/`

Some of the following settings are generally modified according to the analyzed files in order to be compatible with the input file and make the extraction more accurate, such as the entity scope of the community, entity relationship description, and entity merging rules

### 设置工作区变量
首先，让我们确保设置所需的环境变量。有关这些环境变量的详细信息以及可用的环境变量，请参阅变量文档。


这里需要设置平台的api_key,从`https://open.bigmodel.cn/`获取

后面的一些设置为了兼容输入文件，让抽取更加准确，一般根据分析的文件修改，比如社区的实体范围、实体关系说明、实体合并规则

In [4]:
import pandas as pd
import os
from pathlib import Path

os.environ["GRAPHRAG_LLM_API_KEY"] = 'key'

os.environ["GRAPHRAG_LLM_API_BASE"] = 'https://open.bigmodel.cn/api/paas/v4'

os.environ["GRAPHRAG_EMBEDDING_API_BASE"] = 'https://open.bigmodel.cn/api/paas/v4'

os.environ["GRAPHRAG_EMBEDDING_API_KEY"] = 'key'

os.environ["GRAPHRAG_API_KEY"] = 'key'

PIPELINE_YAML = """
encoding_model: cl100k_base
skip_workflows: []
llm:
  api_key: '${GRAPHRAG_API_KEY}'
  type: openai_chat
  model: glm-4-airx
  model_supports_json: false
  temperature: 0.1
  top_p: 0.99
  api_base: 'https://open.bigmodel.cn/api/paas/v4'
  tokens_per_minute: 150000
  requests_per_minute: 500
parallelization:
  stagger: 0.3
async_mode: threaded
embeddings:
  async_mode: threaded
  llm:
    api_key: '${GRAPHRAG_API_KEY}'
    type: openai_embedding
    model: embedding-2
chunks:
  size: 300
  overlap: 100
  group_by_columns:
    - id
    
input:
  type: file # or blob
  file_type: text # or csv
  base_dir: "input"
  file_encoding: utf-8
  file_pattern: .*\.txt$
cache:
  type: file
  base_dir: cache
storage:
  type: file
  base_dir: 'output/${timestamp}/artifacts'
reporting:
  type: file
  base_dir: 'output/${timestamp}/reports'
entity_extraction:
  prompt: prompts/entity_extraction.txt
  entity_types: ["conversation_topic", "emotional_state", "behavior", "intent", "context", "participants"]
  max_gleanings: 0
summarize_descriptions:
  prompt: prompts/summarize_descriptions.txt
  max_length: 500
claim_extraction:
  prompt: prompts/claim_extraction.txt
  description: Any claims or facts that could be relevant to information discovery.
  max_gleanings: 0
community_reports:
  async_mode: asyncio
  llm:
    api_key: '${GRAPHRAG_API_KEY}'
    type: openai_chat
    model: glm-4
    model_supports_json: false
    temperature: 0.1
    top_p: 0.99
    api_base: 'https://open.bigmodel.cn/api/paas/v4'
    tokens_per_minute: 150000
    requests_per_minute: 500
  prompt: prompts/community_report.txt
  max_length: 2000
  max_input_length: 8000
cluster_graph:
  max_cluster_size: 10
embed_graph:
  enabled: false
umap:
  enabled: false
snapshots:
  graphml: false
  raw_entities: false
  top_level_nodes: false
local_search:
  llm_temperature: 0.1
  llm_top_p: 0.99
global_search:
  llm_temperature: 0.1
  llm_top_p: 0.99

"""
pipeline_file =  Path().cwd() /"coplay_analysis_graph"/ "settings.yaml"
with pipeline_file.open("w") as file:
    file.write(PIPELINE_YAML)
pipeline_file

PosixPath('/media/gpt4-pdf-chatbot-langchain/glm-cookbook/demo/graphrag/coplay_analysis_graph/settings.yaml')

In [5]:
 
import os
from pathlib import Path 
community_report_prompt = """

You should always follow the instructions and output a valid JSON object.
The structure of the JSON object you can found in the instructions, use {"answer": "$your_answer"} as the default structure
if you are not sure about the structure.

And you should always end the block with a "```" to indicate the end of the JSON object.

<instructions>
你是一个AI助手，帮助人类分析师进行一般信息发现。信息发现是指在网络中识别和评估与特定实体（例如组织和个人）相关的相关信息的过程。

# 目标
撰写一个社区的综合报告，包括社区的实体列表及其关系和可能的相关声明。该报告将用于向决策者提供关于社区及其潜在影响的相关信息。报告内容包括社区关键实体的概览、法律合规性、技术能力、声誉和值得注意的声明。

# 报告结构

报告应包括以下部分：

- TITLE：代表其关键实体的社区名称 - 标题应简短且具体。可能的话，标题中应包含代表性的命名实体。
- SUMMARY：社区整体结构、其实体之间的关系以及与其实体相关的重要信息的行政摘要。
- IMPACT SEVERITY RATING：0-10之间的浮点分数，代表社区内实体所构成的影响严重性。影响是社区的重要性评分。
- RATING EXPLANATION：对影响严重性评分给出单句解释。
- DETAILED FINDINGS：关于社区的5-10个关键见解的列表。每个见解都应有一个简短的摘要，随后是多段解释性文本，根据以下规则进行论证。要全面。

以以下格式返回输出，格式为良好的JSON格式字符串：
    {{
        "title": <report_title>,
        "summary": <executive_summary>,
        "rating": <impact_severity_rating>,
        "rating_explanation": <rating_explanation>,
        "findings": [
            {{
                "summary":<insight_1_summary>,
                "explanation": <insight_1_explanation>
            }},
            {{
                "summary":<insight_2_summary>,
                "explanation": <insight_2_explanation>
            }}
        ]
    }}

# Grounding Rules

Points supported by data should list their data references as follows:

"This is an example sentence supported by multiple data references [Data: <dataset name> (record ids); <dataset name> (record ids)]."

Do not list more than 5 record ids in a single reference. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:
"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (1), Entities (5, 7); Relationships (23); Claims (7, 2, 34, 64, 46, +more)]."

where 1, 5, 7, 23, 2, 34, 46, and 64 represent the id (not the index) of the relevant data record.

Do not include information where the supporting evidence for it is not provided.


# Example Input
-----------
Text:

Entities

id,entity,description
5,VERDANT OASIS PLAZA,Verdant Oasis Plaza is the location of the Unity March
6,HARMONY ASSEMBLY,Harmony Assembly is an organization that is holding a march at Verdant Oasis Plaza

Relationships

id,source,target,description
37,VERDANT OASIS PLAZA,UNITY MARCH,Verdant Oasis Plaza is the location of the Unity March
38,VERDANT OASIS PLAZA,HARMONY ASSEMBLY,Harmony Assembly is holding a march at Verdant Oasis Plaza
39,VERDANT OASIS PLAZA,UNITY MARCH,The Unity March is taking place at Verdant Oasis Plaza
40,VERDANT OASIS PLAZA,TRIBUNE SPOTLIGHT,Tribune Spotlight is reporting on the Unity march taking place at Verdant Oasis Plaza
41,VERDANT OASIS PLAZA,BAILEY ASADI,Bailey Asadi is speaking at Verdant Oasis Plaza about the march
43,HARMONY ASSEMBLY,UNITY MARCH,Harmony Assembly is organizing the Unity March

Output:
{{
    "title": "Verdant Oasis Plaza and Unity March",
    "summary": "The community revolves around the Verdant Oasis Plaza, which is the location of the Unity March. The plaza has relationships with the Harmony Assembly, Unity March, and Tribune Spotlight, all of which are associated with the march event.",
    "rating": 5.0,
    "rating_explanation": "The impact severity rating is moderate due to the potential for unrest or conflict during the Unity March.",
    "findings": [
        {{
            "summary": "Verdant Oasis Plaza as the central location",
            "explanation": "Verdant Oasis Plaza is the central entity in this community, serving as the location for the Unity March. This plaza is the common link between all other entities, suggesting its significance in the community. The plaza's association with the march could potentially lead to issues such as public disorder or conflict, depending on the nature of the march and the reactions it provokes. [Data: Entities (5), Relationships (37, 38, 39, 40, 41,+more)]"
        }},
        {{
            "summary": "Harmony Assembly's role in the community",
            "explanation": "Harmony Assembly is another key entity in this community, being the organizer of the march at Verdant Oasis Plaza. The nature of Harmony Assembly and its march could be a potential source of threat, depending on their objectives and the reactions they provoke. The relationship between Harmony Assembly and the plaza is crucial in understanding the dynamics of this community. [Data: Entities(6), Relationships (38, 43)]"
        }},
        {{
            "summary": "Unity March as a significant event",
            "explanation": "The Unity March is a significant event taking place at Verdant Oasis Plaza. This event is a key factor in the community's dynamics and could be a potential source of threat, depending on the nature of the march and the reactions it provokes. The relationship between the march and the plaza is crucial in understanding the dynamics of this community. [Data: Relationships (39)]"
        }},
        {{
            "summary": "Role of Tribune Spotlight",
            "explanation": "Tribune Spotlight is reporting on the Unity March taking place in Verdant Oasis Plaza. This suggests that the event has attracted media attention, which could amplify its impact on the community. The role of Tribune Spotlight could be significant in shaping public perception of the event and the entities involved. [Data: Relationships (40)]"
        }}
    ]
}}


# Real Data

Use the following text for your answer. Do not make anything up in your answer.

Text:
{input_text}

The report should include the following sections:

- TITLE: community's name that represents its key entities - title should be short but specific. When possible, include representative named entities in the title.
- SUMMARY: An executive summary of the community's overall structure, how its entities are related to each other, and significant information associated with its entities.
- IMPACT SEVERITY RATING: a float score between 0-10 that represents the severity of IMPACT posed by entities within the community.  IMPACT is the scored importance of a community.
- RATING EXPLANATION: Give a single sentence explanation of the IMPACT severity rating.
- DETAILED FINDINGS: A list of 5-10 key insights about the community. Each insight should have a short summary followed by multiple paragraphs of explanatory text grounded according to the grounding rules below. Be comprehensive.

Return output as a well-formed JSON-formatted string with the following format:
    {{
        "title": <report_title>,
        "summary": <executive_summary>,
        "rating": <impact_severity_rating>,
        "rating_explanation": <rating_explanation>,
        "findings": [
            {{
                "summary":<insight_1_summary>,
                "explanation": <insight_1_explanation>
            }},
            {{
                "summary":<insight_2_summary>,
                "explanation": <insight_2_explanation>
            }}
        ]
    }}

# Grounding Rules

Points supported by data should list their data references as follows:

"This is an example sentence supported by multiple data references [Data: <dataset name> (record ids); <dataset name> (record ids)]."

Do not list more than 5 record ids in a single reference. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:
"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (1), Entities (5, 7); Relationships (23); Claims (7, 2, 34, 64, 46, +more)]."

where 1, 5, 7, 23, 2, 34, 46, and 64 represent the id (not the index) of the relevant data record.

Do not include information where the supporting evidence for it is not provided.

Output:
</instructions>

```json
"""
community_report_file =  Path().cwd() /"coplay_analysis_graph"/ "prompts" / "community_report.txt"
with community_report_file.open("w") as file:
    file.write(community_report_prompt)
community_report_file

PosixPath('/media/gpt4-pdf-chatbot-langchain/glm-cookbook/demo/graphrag/coplay_analysis_graph/prompts/community_report.txt')

In [6]:
 
import os
from pathlib import Path 
entity_extraction_prompt = """
-目标-
给定一个与此活动潜在相关的文本文档和一个实体类型列表，从文本中识别所有这些类型的实体以及所有已识别实体之间的关系。

-步骤-
1. 识别所有实体。对于每个识别的实体，提取以下信息：
- entity_name：实体的名称，首字母大写
- entity_type：以下类型之一：[{entity_types}]
- entity_description：全面描述实体的属性和活动
将每个实体格式化为 ("entity"{tuple_delimiter}<entity_name>{tuple_delimiter}<entity_type>{tuple_delimiter}<entity_description>)

2. 从步骤1中识别的实体中，识别所有 *明确相关* 的 (source_entity, target_entity) 对。
对于每对相关实体，提取以下信息：
- source_entity：在步骤1中识别的源实体名称
- target_entity：在步骤1中识别的目标实体名称
- relationship_description：解释为什么你认为源实体和目标实体彼此相关
- relationship_strength：一个数字分数，表示源实体和目标实体之间关系的强度
将每个关系格式化为 ("relationship"{tuple_delimiter}<source_entity>{tuple_delimiter}<target_entity>{tuple_delimiter}<relationship_description>{tuple_delimiter}<relationship_strength>)

3. 返回英文输出作为所有在步骤1和2中识别的实体和关系的单个列表。使用 **{record_delimiter}** 作为列表分隔符。

4. 完成后，输出 {completion_delimiter}

######################
-示例-
######################
示例 1:

Entity_types: [person, technology, mission, organization, location]
Text:
while Alex clenched his jaw, the buzz of frustration dull against the backdrop of Taylor's authoritarian certainty. It was this competitive undercurrent that kept him alert, the sense that his and Jordan's shared commitment to discovery was an unspoken rebellion against Cruz's narrowing vision of control and order.

Then Taylor did something unexpected. They paused beside Jordan and, for a moment, observed the device with something akin to reverence. “If this tech can be understood..." Taylor said, their voice quieter, "It could change the game for us. For all of us.”

The underlying dismissal earlier seemed to falter, replaced by a glimpse of reluctant respect for the gravity of what lay in their hands. Jordan looked up, and for a fleeting heartbeat, their eyes locked with Taylor's, a wordless clash of wills softening into an uneasy truce.

It was a small transformation, barely perceptible, but one that Alex noted with an inward nod. They had all been brought here by different paths
################
Output:
("entity"{tuple_delimiter}"Alex"{tuple_delimiter}"person"{tuple_delimiter}"Alex is a character who experiences frustration and is observant of the dynamics among other characters."){record_delimiter}
("entity"{tuple_delimiter}"Taylor"{tuple_delimiter}"person"{tuple_delimiter}"Taylor is portrayed with authoritarian certainty and shows a moment of reverence towards a device, indicating a change in perspective."){record_delimiter}
("entity"{tuple_delimiter}"Jordan"{tuple_delimiter}"person"{tuple_delimiter}"Jordan shares a commitment to discovery and has a significant interaction with Taylor regarding a device."){record_delimiter}
("entity"{tuple_delimiter}"Cruz"{tuple_delimiter}"person"{tuple_delimiter}"Cruz is associated with a vision of control and order, influencing the dynamics among other characters."){record_delimiter}
("entity"{tuple_delimiter}"The Device"{tuple_delimiter}"technology"{tuple_delimiter}"The Device is central to the story, with potential game-changing implications, and is revered by Taylor."){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Taylor"{tuple_delimiter}"Alex is affected by Taylor's authoritarian certainty and observes changes in Taylor's attitude towards the device."{tuple_delimiter}7){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Jordan"{tuple_delimiter}"Alex and Jordan share a commitment to discovery, which contrasts with Cruz's vision."{tuple_delimiter}6){record_delimiter}
("relationship"{tuple_delimiter}"Taylor"{tuple_delimiter}"Jordan"{tuple_delimiter}"Taylor and Jordan interact directly regarding the device, leading to a moment of mutual respect and an uneasy truce."{tuple_delimiter}8){record_delimiter}
("relationship"{tuple_delimiter}"Jordan"{tuple_delimiter}"Cruz"{tuple_delimiter}"Jordan's commitment to discovery is in rebellion against Cruz's vision of control and order."{tuple_delimiter}5){record_delimiter}
("relationship"{tuple_delimiter}"Taylor"{tuple_delimiter}"The Device"{tuple_delimiter}"Taylor shows reverence towards the device, indicating its importance and potential impact."{tuple_delimiter}9){completion_delimiter}
#############################
-Real Data-
######################
Entity_types: {entity_types}
Text: {input_text}
######################
Output:
"""
entity_extraction_file =  Path().cwd() /"coplay_analysis_graph"/ "prompts" / "entity_extraction.txt"
with entity_extraction_file.open("w") as file:
    file.write(entity_extraction_prompt)
entity_extraction_file

PosixPath('/media/gpt4-pdf-chatbot-langchain/glm-cookbook/demo/graphrag/coplay_analysis_graph/prompts/entity_extraction.txt')

In [7]:
 
import os
from pathlib import Path 
claim_extraction_prompt = """

-Target activity-
You are an intelligent assistant that helps a human analyst to analyze claims against certain entities presented in a text document.

-目标-
给定一个与此活动潜在相关的文本文件，一个实体规范和一个指控描述，提取所有符合实体规范的实体及所有针对这些实体的指控。

-步骤-
1. 提取所有符合预定义实体规范的命名实体。实体规范可以是实体名称列表或实体类型列表。
2. 对于步骤1中识别的每个实体，提取与该实体相关的所有指控。指控需要符合指定的指控描述，并且实体应为指控的主体。
对于每个指控，提取以下信息：
- 主体：指控的主体实体名称，全部大写。主体实体是指控中描述的行为的实施者。主体需要是步骤1中识别的命名实体之一。
- 客体：指控的客体实体名称，全部大写。客体实体是报告/处理或受到指控中描述行为影响的实体。如果客体实体未知，使用 **NONE**。
- 指控类型：指控的总体类别，全部大写。命名方式应在多个文本输入中重复使用，以便相似指控共享相同的指控类型。
- 指控状态：**TRUE**、**FALSE** 或 **SUSPECTED**。TRUE 表示指控得到确认，FALSE 表示指控被发现为假，SUSPECTED 表示指控未验证。
- 指控描述：详细描述解释指控背后的推理，以及所有相关的证据和参考资料。
- 指控日期：指控提出的期间（start_date, end_date）。start_date 和 end_date 都应为 ISO-8601 格式。如果指控是在单一日期提出的，则将同一日期设为 start_date 和 end_date。如果日期未知，则返回 **NONE**。
- 指控来源文本：列出所有与指控相关的原始文本引文。

将每个指控格式化为 (<subject_entity>{tuple_delimiter}<object_entity>{tuple_delimiter}<claim_type>{tuple_delimiter}<claim_status>{tuple_delimiter}<claim_start_date>{tuple_delimiter}<claim_end_date>{tuple_delimiter}<claim_description>{tuple_delimiter}<claim_source>)

3. 返回步骤1和2中识别的所有指控的英文单一列表。使用 **{record_delimiter}** 作为列表分隔符。

4. 完成时输出 {completion_delimiter}

-Examples-
Example 1:
Entity specification: organization
Claim description: red flags associated with an entity
Text: According to an article on 2022/01/10, Company A was fined for bid rigging while participating in multiple public tenders published by Government Agency B. The company is owned by Person C who was suspected of engaging in corruption activities in 2015.
Output:

(COMPANY A{tuple_delimiter}GOVERNMENT AGENCY B{tuple_delimiter}ANTI-COMPETITIVE PRACTICES{tuple_delimiter}TRUE{tuple_delimiter}2022-01-10T00:00:00{tuple_delimiter}2022-01-10T00:00:00{tuple_delimiter}Company A was found to engage in anti-competitive practices because it was fined for bid rigging in multiple public tenders published by Government Agency B according to an article published on 2022/01/10{tuple_delimiter}According to an article published on 2022/01/10, Company A was fined for bid rigging while participating in multiple public tenders published by Government Agency B.)
{completion_delimiter}

Example 2:
Entity specification: Company A, Person C
Claim description: red flags associated with an entity
Text: According to an article on 2022/01/10, Company A was fined for bid rigging while participating in multiple public tenders published by Government Agency B. The company is owned by Person C who was suspected of engaging in corruption activities in 2015.
Output:

(COMPANY A{tuple_delimiter}GOVERNMENT AGENCY B{tuple_delimiter}ANTI-COMPETITIVE PRACTICES{tuple_delimiter}TRUE{tuple_delimiter}2022-01-10T00:00:00{tuple_delimiter}2022-01-10T00:00:00{tuple_delimiter}Company A was found to engage in anti-competitive practices because it was fined for bid rigging in multiple public tenders published by Government Agency B according to an article published on 2022/01/10{tuple_delimiter}According to an article published on 2022/01/10, Company A was fined for bid rigging while participating in multiple public tenders published by Government Agency B.)
{record_delimiter}
(PERSON C{tuple_delimiter}NONE{tuple_delimiter}CORRUPTION{tuple_delimiter}SUSPECTED{tuple_delimiter}2015-01-01T00:00:00{tuple_delimiter}2015-12-30T00:00:00{tuple_delimiter}Person C was suspected of engaging in corruption activities in 2015{tuple_delimiter}The company is owned by Person C who was suspected of engaging in corruption activities in 2015)
{completion_delimiter}

-Real Data-
Use the following input for your answer.
Entity specification: {entity_specs}
Claim description: {claim_description}
Text: {input_text}
Output:
"""
claim_extraction_file =  Path().cwd() /"coplay_analysis_graph"/ "prompts" / "claim_extraction.txt"
with claim_extraction_file.open("w") as file:
    file.write(claim_extraction_prompt)
claim_extraction_file

PosixPath('/media/gpt4-pdf-chatbot-langchain/glm-cookbook/demo/graphrag/coplay_analysis_graph/prompts/claim_extraction.txt')

#### Run the index creation process
Finally, we will run the index creation process

#### 运行索引创建流程
最后我们将运行索引创建流程



In [8]:
!mkdir -p $root_dir/input/ && cp -r input_coplay_analysis_graph.txt $root_dir/input/


In [9]:

!python -m graphrag.index --root "$root_dir"

Logging enabled at r 
/media/gpt4-pdf-chatbot-langchain/glm-cookbook/demo/graphrag/coplay_analysis_gra
ph/output/20240929-215737/reports/indexing-engine.log
⠙ GraphRAG Indexer 
⠙ GraphRAG Indexer les loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠙ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠙ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠙ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
└── create_base_text_units
⠴ GraphRAG Indexer ━━━━━━━━━━━━━━━━━   0% -:--:-- 0:00:00
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
└── create_base_text_units
⠧ GraphRAG Indexer ━━━━━━━━━━━━━━━━━   0% -:--:-- 0:00:01
├── Loading Input 